In [2]:
# !pip install torch
# !pip install langchain
# !pip install langchain-community
# !pip install pymupdf
# !pip install chromadb
# !pip install transformers
# !pip install sentence-transformers
!pip install rank_bm25

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import Chroma 
from langchain_community.retrievers import BM25Retriever  # 추가: BM25Retriever
from langchain.retrievers import EnsembleRetriever  # 추가: EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# 단계 1: 문서 로드(Load Documents)
PDF_FILE_PATH = "SPRi AI Brief 5월호 산업동향.pdf"
try:
    loader = PyMuPDFLoader(PDF_FILE_PATH)
    docs = loader.load()
except Exception as e:
    print(f"Error loading PDF: {e}")
    print(f"Please ensure the PDF file is available at '{PDF_FILE_PATH}' or provide a direct downloadable URL.")
    docs = [] 

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = HuggingFaceEmbeddings(
    model_name="BM-K/KoSimCSE-roberta-multitask"
)

# 단계 4: DB 생성(Create DB) 및 저장
vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings,
    persist_directory="./chroma_db",
    collection_metadata={"hnsw:construction_ef": 500, "hnsw:M": 32},
)

# 단계 5: 검색기(Retriever) 생성
# 변경: BM25Retriever와 Chroma 검색기를 결합한 EnsembleRetriever 사용
# BM25Retriever로 키워드 기반 검색 추가, Chroma로 임베딩 기반 검색 유지
chroma_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        'k': 3,
        'fetch_k': 20,
        'score_threshold': 0.6
    }
)
bm25_retriever = BM25Retriever.from_documents(
    documents=split_documents,
    k=3  # 추가: BM25 검색 결과 상위 3개 문서 반환
)
ensemble_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, bm25_retriever],
    weights=[0.5, 0.5]  # 추가: Chroma와 BM25 결과에 각각 50% 가중치 부여
)

# 단계 6: 프롬프트 생성(Create Prompt)
system_prompt_text = """당신은 주어진 컨텍스트를 기반으로 질문에 답변하는 AI 어시스턴트입니다.
답변은 간결하고 정확해야 하며, 한국어로 작성하세요.
컨텍스트에 없는 정보는 절대로 사용하지 말고, 만약 컨텍스트에 답변이 없다면 "컨텍스트에서 답변을 찾을 수 없습니다."라고 솔직하게 답변하세요."""

llama3_prompt_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id>

#Context:
{context}

#Question:
{question}<|eot_id|><|start_header_id|>assistant<|end_header_id>
"""

prompt = PromptTemplate(
    template=llama3_prompt_template,
    input_variables=["context", "question"],
    partial_variables={"system_prompt": system_prompt_text}
)

# 단계 7: 언어모델(LLM) 생성
model_name = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
    )
except Exception as e:
    print(f"Error loading model {model_name}: {e}")
    raise e

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token 

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    return_full_text=False, 
    max_new_tokens=256,  # 변경: 256 -> 1024로 증가하여 더 긴 답변 생성
    repetition_penalty=1.2,  # 추가: 긴 답변에서 반복 억제 강화
    top_p=0.95  # 변경: 0.9 -> 0.95로 조정하여 긴 답변에서도 다양성 유지
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": ensemble_retriever | RunnableLambda(format_docs), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.43it/s]
Device set to use cuda:0


In [10]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "EU집행위원회는 AI에 얼마를 투자했지?"
response = chain.invoke(question)
print(response)

/home/elicer/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


EU 집행위원회는 'InvestAI' 이니셔티브를 통해 AI에 총 2,000억 유로를 투자했습니다. 또한, AI 대륙 행동계획을 발표하면서 AI 컴퓨팅 인프라 구축, 데이터 접근성 확대, 전략적 영역의 AI 촉진, AI 역량과 인재 육성, AI 법 시행 간소화 등의 5가지 영역을 중점 추진합니다. AI 대륙 행동계획에는 AI 컴퓨팅 인프라와 데이터 접근성 확보 및 AI 도입 확대를 추진하는데 200억 유로는 AI 기가팩토리의 구축을 위해서 사용됩니다. AI 기가팩터리는 클라우드와 데이터센터에 대한 민간 투자를 활성화하고 있습니다. AI 법 시행 간소화를 위해서는 'EU 클라우드 및 AI 개발법'을 입법하여 민간 투자를 촉진합니다. AI 대륙 행동계획에는 AI 컴퓨팅 인프라 구축, 데이터 접근성 확대, 전략적 영역의 AI 촉진, AI 역량과 인재 육성, AI 법 시행 간소화 등을


In [11]:
question = "구글이 발표한 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜은?"
response = chain.invoke(question)
print(response)

/home/elicer/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Google이 발표한 에이전트 간 상호운용성을 보장하기 위한 개방형 통신 프로토콜은 'A2A'입니다. A2A는 에이전트 간 기능 탐색, 작업 관리, 협업, 사용자 경험 협의 등의 다양한 기능을 지원합니다. 또한, A2A는 제미나이 모델과 SDK에서 앤스로픽의 MCP 지원을 추가했습니다. A2A는 HTTP, SSE, JSON-RPC 등의 기존 표준을 기반으로 구축됐습니다. A2A는 작업을 구성하고 전달하는 역할을 하는 클라이언트 에이전트와 작업을 수행하는 원격 에이전트 간 원활한 통신을 위해 다양한 기능을 제공합니다. A2A는 에이전트가 자신의 기능을 JSON 형식의 ‘에이전트 카드’를 통해 공개하면 클라이언트 에이전트는 작업 수행에 가장 적합한 에이전트를 식별해 A2A로 원격 에이전트와 통신합니다. 이렇듯 A2A는 에이전트 간 협업을 위한 상위 레벨의 프로토콜입니다. 




In [5]:
# !rm -rf chroma_db